In [ ]:
import numpy as np
import os.path as osp
import os
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 
import sys

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)

from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

from hidden_points_removal import hidden_point_removal_o3d
from utils import get_pcd
from point_cloud_utils import project_pcd, filter_points_from_dict
from point_to_pixels import point_to_pixel
from visualization_utils import unite_pcd_and_img

In [ ]:
DATASET_PATH = os.path.join('/Users/laurenzheidrich/Downloads/','fused_dataset')
SEQUENCE_NUM = 10

config_filtered = KittiOdometryDatasetConfig(
    cache=True,
    dataset_path=DATASET_PATH,
    correct_scan_calibration=True,
    filters=FilterList(
        [
            KittiGTMovingObjectFilter(
                os.path.join(
                    DATASET_PATH,
                    "sequences",
                    "%.2d" % SEQUENCE_NUM,
                    "labels",
                )
            ),
            RangeFilter(2.5, 120),
            ApplyPose(),
        ]
    ),
)

dataset = KittiOdometryDataset(config_filtered, SEQUENCE_NUM)

In [ ]:
points_index = 20
cam_name = "cam2"

pcd_o3d = get_pcd(dataset.get_point_cloud(points_index))
pcd = np.asarray(pcd_o3d.points)

image_PIL = dataset.get_image(cam_name, points_index)

image = cv2.cvtColor(np.array(image_PIL), cv2.COLOR_RGB2BGR)

In [ ]:
calib_dict = dataset.get_calib_dict()
T_lidar2cam = dataset.project_velo_to_cam(calib_dict, cam_name)

In [ ]:
pcd_camframe = project_pcd(pcd, T_lidar2cam)

hpr_mask = hidden_point_removal_o3d(pcd_camframe, camera=[0,0,1.73])
pcd_camframe_hpr = pcd_camframe[hpr_mask]
pcd_hpr = pcd[hpr_mask]

In [ ]:
# Here I need to check why pcd is needed and what effect that has on the result
point_to_pixel_dict = point_to_pixel(pcd_camframe, pcd, calib_dict, image.shape[0], image.shape[1])

point_to_pixel_dict_hpr = point_to_pixel(pcd_camframe_hpr, pcd_hpr, calib_dict, image.shape[0], image.shape[1])

In [ ]:
pcd_camframe_fov = filter_points_from_dict(pcd, point_to_pixel_dict)
pcd_camframe_fov_hpr = filter_points_from_dict(pcd_hpr, point_to_pixel_dict_hpr)

In [ ]:
print(len(point_to_pixel_dict))
print(len(point_to_pixel_dict_hpr))
print(pcd_camframe_fov.shape)
print(pcd_camframe_fov_hpr.shape)

In [ ]:
img_overlay = unite_pcd_and_img(point_to_pixel_dict, pcd_camframe, image)

img_overlay_hpr = unite_pcd_and_img(point_to_pixel_dict_hpr, pcd_camframe_hpr, image)

In [ ]:
# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1)

# Display images in subplots
ax1.imshow(img_overlay)
ax2.imshow(img_overlay_hpr)

# Hide the axes ticks and labels
ax1.set_xticks([]), ax1.set_yticks([])
ax2.set_xticks([]), ax2.set_yticks([])

plt.show()

In [ ]:
vis_pcd = o3d.geometry.PointCloud()
vis_pcd.points = o3d.utility.Vector3dVector(pcd_camframe_fov)

o3d.visualization.draw_geometries([vis_pcd])
